# **Prueba Técnica Análisis y Limpieza de Datos de FIFA 19**

# **Autor:** Sebastián Peña Pérez - Data Analyst (Asistido por IA)
# **Rol:** Data Analyst para Crack the Code
# **Objetivo:** Realizar la limpieza, pre-procesamiento y transformación del dataset de FIFA 19 para preparar los datos para un análisis exploratorio y visualización en Power BI.


In [ ]:
#Configuración e importación de librerías

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns




In [52]:
#Carga del dataset

df = pd.read_csv("C:/Users/sebas/OneDrive - Universidad Autonoma de Occidente/HDV 2025/Prueba técnica Crack the Code - Data Analyst/kl.csv", encoding='windows-1252' )

print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")

df = df.drop(columns=['Unnamed: 0'])


Dataset cargado: 18207 filas, 89 columnas


In [53]:
#Exploración inicial 

print("\n" + "="*50)
print("EXPLORACIÓN INICIAL DEL DATASET")
print("="*50)

# Mostrar primeras filas
print("\n📋 PRIMERAS 5 FILAS:")
print(df.head())

# Información general del dataset
print("\n📊 INFORMACIÓN GENERAL:")
print(df.info())


# Verificar valores nulos
print("\n❌ VALORES NULOS POR COLUMNA:")
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0].sort_values(ascending=False))



EXPLORACIÓN INICIAL DEL DATASET

📋 PRIMERAS 5 FILAS:
       ID               Name   Age  \
0  158023           L. Messi  31.0   
1   20801  Cristiano Ronaldo  33.0   
2  190871          Neymar Jr  26.0   
3  193080             De Gea  27.0   
4  192985       K. De Bruyne  27.0   

                                            Photo Nationality  \
0  https://cdn.sofifa.org/players/4/19/158023.png   Argentina   
1   https://cdn.sofifa.org/players/4/19/20801.png    Portugal   
2  https://cdn.sofifa.org/players/4/19/190871.png      Brazil   
3  https://cdn.sofifa.org/players/4/19/193080.png       Spain   
4  https://cdn.sofifa.org/players/4/19/192985.png     Belgium   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.org/flags/52.png     94.0         94   
1  https://cdn.sofifa.org/flags/38.png     94.0         94   
2  https://cdn.sofifa.org/flags/54.png     92.0         93   
3  https://cdn.sofifa.org/flags/45.png     91.0         93   
4   https://cdn.

In [ ]:
#4 limpieza y preparación de datos


#Función 1: conversión de moneda, Convierte moneda (€110.5M, €500K) a float en millones o miles de euros

def convert_currency_to_millions(value):
    """Convierte string monetario a float en millones de euros."""
    if not isinstance(value, str):
        return np.nan
    
    # Remover símbolo de euro y espacios
    clean_value = value.replace('€', '').strip()
    
    # Conversión según sufijo
    if clean_value.endswith('M'):
        return float(clean_value[:-1])
    elif clean_value.endswith('K'):
        return float(clean_value[:-1]) / 1000
    
    return np.nan

# Aplicar conversiones monetarias
currency_columns = {'Value': 'Value_M', 'Wage': 'Wage_M', 'Release Clause': 'Release_Clause_M'}

for old_col, new_col in currency_columns.items():
    if old_col in df.columns:
        df[new_col] = df[old_col].apply(convert_currency_to_millions)

# Limpiar columnas originales
df.drop(list(currency_columns.keys()), axis=1, inplace=True, errors='ignore')
print("✓ Columnas monetarias convertidas a millones de euros")



In [55]:
#Función 2: Altua, convierte pulgadas y pies a centímetros  y libras a kilogramos

def convert_height_to_cm(height_str):
    """Convierte altura de pies'pulgadas a centímetros."""
    if not isinstance(height_str, str) or "'" not in height_str:
        return np.nan
    
    parts = height_str.split("'")
    if len(parts) != 2:
        return np.nan
    
    try:
        feet, inches = int(parts[0]), int(parts[1])
        return (feet * 12 + inches) * 2.54
    except ValueError:
        return np.nan

def convert_weight_to_kg(weight_str):
    """Convierte peso de lbs a kg."""
    if not isinstance(weight_str, str):
        return np.nan
    
    try:
        lbs = int(weight_str.replace('lbs', ''))
        return lbs * 0.453592
    except ValueError:
        return np.nan

# Aplicar conversiones físicas
if 'Height' in df.columns:
    df['Height_cm'] = df['Height'].apply(convert_height_to_cm)
    df.drop('Height', axis=1, inplace=True)

if 'Weight' in df.columns:
    df['Weight_kg'] = df['Weight'].apply(convert_weight_to_kg)
    df.drop('Weight', axis=1, inplace=True)

print("✓ Medidas físicas convertidas a sistema métrico")




✓ Medidas físicas convertidas a sistema métrico


In [56]:
df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Height_cm,Weight_kg
0,158023,L. Messi,31.0,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94.0,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M,170.18,72.121128
1,20801,Cristiano Ronaldo,33.0,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94.0,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M,187.96,83.007336
2,190871,Neymar Jr,26.0,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92.0,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M,175.26,68.038800
3,193080,De Gea,27.0,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91.0,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M,193.04,76.203456
4,192985,K. De Bruyne,27.0,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91.0,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M,180.34,69.853168


###Etapa 5 Análisis de Arquetipos de jugadores

In [1]:
# %% [markdown]
# # Análisis de Datos de Jugadores de FIFA 19
# ---
# ## Etapa 5: Análisis Avanzado - Identificación de Arquetipos de Jugadores Atípicos
#
# En esta etapa, vamos más allá de los outliers estadísticos básicos. Nuestro objetivo es identificar perfiles de jugadores estratégicamente valiosos que no son evidentes a primera vista. Crearemos métricas compuestas y aplicaremos una lógica de negocio para encontrar "Especialistas", "Talentos Infravalorados", "Diamantes en Bruto", "Maestros Veteranos" y "Anomalías Físicas".

# %%
# Importamos librerías adicionales si son necesarias
from scipy.stats import zscore

# Se asume que 'datos_df' es el DataFrame limpio de las etapas anteriores.
# Si estás continuando en el mismo notebook, ya debería estar en memoria.
print("Iniciando Etapa 5: Análisis de Arquetipos.")
print(f"Dataset de entrada tiene {datos_df.shape[0]} filas.")

# %% [markdown]
# ### 5.1 Ingeniería de Métricas Avanzadas
#
# Primero, creamos los índices personalizados que nos servirán para identificar los arquetipos.

# %%
# --- 1. Índice de Rendimiento por Posición ---
# Definimos las habilidades más importantes para cada rol
habilidades_portero = ['Reflejos_Portero', 'Estirada_Portero', 'Parada_Portero', 'Posicion_Portero', 'Compostura']
habilidades_defensor = ['Entrada_Normal', 'Marcaje', 'Intercepciones', 'Fuerza', 'Precision_Cabeza', 'Reacciones']
habilidades_mediocampista = ['Pase_Corto', 'Vision', 'Control_Balon', 'Regates', 'Compostura', 'Resistencia']
habilidades_atacante = ['Finalizacion', 'Posicionamiento', 'Potencia_Tiro', 'Control_Balon', 'Reacciones', 'Compostura']

# Calculamos el índice como el promedio de estas habilidades clave
def calcular_indice_rendimiento(jugador):
    if jugador['Grupo_Posicion'] == 'Portero':
        return jugador[habilidades_portero].mean()
    elif jugador['Grupo_Posicion'] == 'Defensor':
        return jugador[habilidades_defensor].mean()
    elif jugador['Grupo_Posicion'] == 'Mediocampista':
        return jugador[habilidades_mediocampista].mean()
    elif jugador['Grupo_Posicion'] == 'Atacante':
        return jugador[habilidades_atacante].mean()
    return 0

datos_df['Indice_Rendimiento'] = datos_df.apply(calcular_indice_rendimiento, axis=1)
print("Creada la columna 'Indice_Rendimiento'.")

# --- 2. Ratio de Valor por Rendimiento (Métrica 'Moneyball') ---
# Sumamos 1 al denominador para evitar divisiones por cero con jugadores sin salario/valor
datos_df['Ratio_Valor_Rendimiento'] = datos_df['Indice_Rendimiento'] / (datos_df['Salario'] + datos_df['Valor_Mercado'] + 1)
print("Creada la columna 'Ratio_Valor_Rendimiento'.")

# --- 3. Z-Score de Atributos Físicos por Posición ---
# Calculamos cuán atípico es un jugador físicamente comparado con otros de su misma posición
columnas_fisicas = ['Altura_cm', 'Peso_kg', 'Fuerza', 'Velocidad_Sprint']
for col in columnas_fisicas:
    # El Z-score se calcula dentro de cada grupo de posición
    datos_df[f'ZScore_{col}'] = datos_df.groupby('Grupo_Posicion')[col].transform(lambda x: zscore(x, nan_policy='omit'))
print("Creadas las columnas de Z-Score para atributos físicos.")

# %% [markdown]
# ### 5.2 Identificación y Etiquetado de Arquetipos
#
# Ahora aplicamos los filtros para encontrar a los jugadores que encajan en cada perfil.

# %%
# Inicializamos una columna para almacenar los arquetipos de cada jugador
datos_df['Arquetipo_Atipico'] = [[] for _ in range(len(datos_df))]

def agregar_arquetipo(df, condicion, nombre_arquetipo):
    """Función auxiliar para añadir etiquetas de arquetipo a los jugadores que cumplen una condición."""
    indices = df[condicion].index
    for i in indices:
        df.loc[i, 'Arquetipo_Atipico'].append(nombre_arquetipo)

# --- Arquetipo 1: El Especialista Oculto ---
habilidades_especialista = ['Precision_Tiro_Libre', 'Pase_Largo', 'Centros', 'Entrada_Agresiva']
cond_valoracion = (datos_df['Valoracion_General'] >= 70) & (datos_df['Valoracion_General'] <= 82)
for habilidad in habilidades_especialista:
    cond_habilidad = datos_df[habilidad] > datos_df[habilidad].quantile(0.98)
    agregar_arquetipo(datos_df, cond_valoracion & cond_habilidad, 'Especialista Oculto')

# --- Arquetipo 2: El Talento Infravalorado (Moneyball) ---
cond_moneyball = datos_df['Ratio_Valor_Rendimiento'] > datos_df['Ratio_Valor_Rendimiento'].quantile(0.99)
agregar_arquetipo(datos_df, cond_moneyball, 'Talento Infravalorado')

# --- Arquetipo 3: El Diamante en Bruto ---
cond_diamante = (datos_df['Edad'] < 21) & \
                (datos_df['Potencial'] > 87) & \
                (datos_df['Aceleracion'] > datos_df['Aceleracion'].quantile(0.90)) & \
                (datos_df['Compostura'] < datos_df['Compostura'].quantile(0.40))
agregar_arquetipo(datos_df, cond_diamante, 'Diamante en Bruto')

# --- Arquetipo 4: El Maestro Veterano ---
cond_maestro = (datos_df['Edad'] > 33) & \
               (datos_df['Resistencia'] < datos_df['Resistencia'].quantile(0.40)) & \
               ((datos_df['Vision'] > 85) | (datos_df['Compostura'] > 85))
agregar_arquetipo(datos_df, cond_maestro, 'Maestro Veterano')

# --- Arquetipo 5: La Anomalía Física ---
# Jugadores extremadamente altos o bajos para su posición (Z-score > 3 o < -3)
cond_anomalia_fisica = (datos_df['ZScore_Altura_cm'].abs() > 3) | (datos_df['ZScore_Fuerza'].abs() > 3)
agregar_arquetipo(datos_df, cond_anomalia_fisica, 'Anomalía Física')

print("Etiquetado de arquetipos completado.")

# %% [markdown]
# ### 5.3 Creación del DataFrame Final de Jugadores Atípicos
#
# Filtramos solo los jugadores que fueron etiquetados con al menos un arquetipo y creamos nuestro entregable final.

# %%
# Filtramos el DataFrame para quedarnos solo con los jugadores que tienen alguna etiqueta de arquetipo
# El método .str.len() sobre la lista en la columna nos permite saber cuántas etiquetas tiene
jugadores_atipicos_df = datos_df[datos_df['Arquetipo_Atipico'].str.len() > 0].copy()

print(f"Se ha creado un DataFrame con {len(jugadores_atipicos_df)} jugadores atípicos que encajan en uno o más arquetipos.")

# Seleccionamos las columnas más relevantes para este análisis
columnas_relevantes = [
    'Nombre', 'Edad', 'Club', 'Grupo_Posicion', 'Valoracion_General', 'Potencial',
    'Valor_Mercado', 'Salario', 'Arquetipo_Atipico', 'Indice_Rendimiento',
    'Ratio_Valor_Rendimiento', 'ZScore_Altura_cm', 'ZScore_Fuerza'
]

jugadores_atipicos_df = jugadores_atipicos_df[columnas_relevantes]

# Guardamos el resultado en un CSV para Power BI
jugadores_atipicos_df.to_csv('jugadores_atipicos.csv', index=False, encoding='utf-8-sig')

print("DataFrame 'jugadores_atipicos.csv' guardado exitosamente.")
display(jugadores_atipicos_df.head())



Iniciando Etapa 5: Análisis de Arquetipos.


NameError: name 'datos_df' is not defined